# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objs as go
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

from IPython.display import Video


In [2]:
data_promos_original = pd.read_excel("14. Ongoing promos and placement by retailer USC CONFIDENTIAL.xlsx",
                         'In store placement and promo',skiprows=[0,0],parsedates=["WK Start","Wk End"])

# Data Preparation

In [3]:
data_promos = data_promos_original.copy()

In [4]:
# Summarize Product names
# Remove CT details from a duplicate Product column and call it "Product Summarized"

data_promos.insert(2, 'Product Summarized', data_promos['Product'].str.split(' CT',expand=True)[0])

In [5]:
# Summarize Geography
# Consolidate multiple branches into one

data_promos.insert(1, 'Geography Summarized', data_promos['Geography'])

search = ["KR","ABSCO","ADUSA","Four B","HAC","SEG","The Save Mart"]

for s in search:
    data_promos[s] = data_promos['Geography Summarized'].str.startswith(s, na = False) 
    data_promos.loc[data_promos[s] == True, 'Geography Summarized'] = s
    
data_promos.drop(data_promos.loc[:, 'KR':'The Save Mart'], inplace = True, axis = 1) 

In [6]:
# Summarize Secondary
# Consolidate 3 columns into 1 and add another column for secondary type

data_promos.insert(9, 'Secondary', np.NaN)
data_promos.insert(10, 'Secondary Type', np.NaN)

secondarylist = ['Secondary Deli', 'Secondary Bakery', 'Secondary Center store']

for s in secondarylist:
    data_promos.loc[data_promos[s] == 'Yes', 'Secondary'] = 'Yes'
    data_promos.loc[data_promos[s] == 'Yes', 'Secondary Type'] = s

In [7]:
# Replace empty or NaN cells with "No" for Holiday and Promotion Columns

columns = ['Chicken Program/Meal Deal', 'Secondary',
       'Secondary Type', 'Secondary Deli', 'Secondary Bakery',
       'Secondary Center store', 'Retail Coverage']

data_promos[columns] = data_promos[columns].fillna('No')

data_promos['Holiday']=data_promos['Holiday'].fillna('None')

In [8]:
# Add columns for Year and Month

data_promos.insert(7, 'Month', data_promos['Wk End'].dt.month)
data_promos.insert(8, 'Year', data_promos['Wk End'].dt.year)

In [9]:
data_promos[columns] = data_promos[columns].fillna('No')
data_promos[columns] = data_promos[columns].replace("Yes ", "Yes")
data_promos[columns] = data_promos[columns].replace("yes", "Yes")

data_promos.insert(8, 'Holiday B', 1)
data_promos.loc[data_promos['Holiday'] == 'None', 'Holiday B'] = 0

# Dash Chart Code

In [14]:
fig = go.Figure()
fig1 = go.Figure()
fig2 = go.Figure()

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df=data_promos
df['Year']=df['Wk End'].dt.year
df['Month']=df['Wk End'].dt.month_name(locale = 'English') 
df['mth']=df['Wk End'].dt.month

years=[2018,2019,2020]
platform=df['Platform'].unique()
Product=df['Product'].unique()
retailers=df['Geography Summarized'].unique()
displays=['Secondary Deli','Secondary Bakery','Secondary Center store',"Chicken Program/Meal Deal"]
kpi=['Volume Sales','Unit Sales','Dollar Sales','Volume per $MM ACV',
     'Volume per Store Selling']#'Price per Unit','Average Weekly ACV Distribution'
causations=['No Selection','Any Merch','Any Feature','Any Price Reduction','Any Display']
disclaimer=f'For 2020, data was available till July'
all_options={}
for p in causations:
    if p =='No Selection':
        all_options[p]=kpi
    else:
        all_options[p]=kpi[0:4]


app.layout = html.Div([
        html.Div([
#Year Dropdown
        html.Div([ html.Label(["Year",
            dcc.Dropdown(
                id='year',
                options=[{'label': y, 'value': y} for y in years],
                value=2020
            )])

        ],style={'width': '10%', 'display': 'inline-block'}),

#Platform Dropdown
        html.Div([ html.Label(["Platform",
            dcc.Dropdown(
                id='platform',
                options=[{'label': p, 'value': p} for p in platform],
                value='Dinner Rolls'
            )])

        ],style={'width': '20%', 'display': 'inline-block'}),
#In-store Promo
        html.Div([ html.Label(["In-Store Promo",
            dcc.Dropdown(
                id='promo',
                options=[{'label': c, 'value': c} for c in causations],
                value='No Selection'
            )])

        ],style={'width': '20%', 'display': 'inline-block'}),
             
#KPI
        html.Div([ html.Label(["KPI",
            dcc.Dropdown(
                id='kpi'
            )])

        ],style={'width': '20%', 'display': 'inline-block'})
        ]),
    html.Div(children=disclaimer, style={
        'textAlign': 'left','bold':True,
        'color': 'blue'
    }),


    html.Div([dcc.Graph(
        id='graph2',
        figure=fig1
        )],style={'width': '50%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(
        id='graph3',
        figure=fig1
        )],style={'width': '50%', 'display': 'inline-block'}),
    html.Div([dcc.Graph(
        id='graph',
        figure=fig
        )])
])

@app.callback(
    Output('kpi', 'options'),
    [Input('promo', 'value')])
def set_promo_options(selected_promo):
    return [{'label': i, 'value': i} for i in all_options[selected_promo]]



@app.callback(
    Output('kpi', 'value'),
    [Input('kpi', 'options')])
def set_cities_value(available_options):
    return available_options[0]['value']


@app.callback(
    [Output('graph', 'figure'),Output('graph2', 'figure'),Output('graph3', 'figure')],
    [Input('year', 'value'),
     Input('platform', 'value'),Input('promo', 'value'),Input('kpi', 'value')])
    
def update_graph(year_value,platform_value,promo_value,kpi_value):
    if promo_value == 'No Selection':
        kpi=kpi_value
    else:
        kpi=kpi_value+' '+promo_value
    fig = go.Figure()
    fig1 = go.Figure()
    fig2 = go.Figure()
    
    dff = df.loc[(df['Year'] == year_value) & (df['Platform'] == platform_value)]
    df2 = df.loc[df['Platform'] == platform_value]
        
    for d in displays:
        
        if promo_value == 'No Selection':
            dff1=dff.groupby(['Wk End','Platform',d]).agg({kpi:'sum'}).reset_index()
            fig.add_trace(go.Scatter(x=dff1[dff1[d] == 'Yes']['Wk End'],y=dff1[dff1[d] == 'Yes'][kpi]
                                     ,mode='lines+markers',name=d))
            xaxisTitle='Weekend Date'
            titletext=f'Weekly Total {kpi_value}'
            kff2 = dff.groupby(['Wk End']).agg({'Holiday B':'sum'}).reset_index()
            kff3 = dff[dff['Secondary Deli']=='Yes']
            kff4 = dff[dff['Secondary Bakery']=='Yes']
            kff5 = dff[dff['Secondary Center store']=='Yes']
            kff3 = kff3.groupby(['Wk End']).agg({kpi:'sum'})
            kff4 = kff4.groupby(['Wk End']).agg({kpi:'sum'})
            kff5 = kff5.groupby(['Wk End']).agg({kpi:'sum'})
            ymin = min([kff3.min().sum(), kff4.min().sum(), kff5.min().sum()])
            ymax = max([kff3.max().sum(), kff4.max().sum(), kff5.max().sum()])

            for i in kff2.index:
                if kff2['Holiday B'][i] > 0:
                    if year_value == 2020:
                        fig.add_shape(type="line",
                                         x0 = kff2['Wk End'][i],
                                         x1 = kff2['Wk End'][i],
                                         y0 = ymin,
                                         y1 = ymax,
                                         line = dict(color="orange",width=42),
                                         opacity=0.1,
                                         layer="below")
                    else:
                        fig.add_shape(type="line",
                                         x0 = kff2['Wk End'][i],
                                         x1 = kff2['Wk End'][i],
                                         y0 = ymin,
                                         y1 = ymax,
                                         line = dict(color="orange",width=20),
                                         opacity=0.1,
                                         layer="below")                        
        else:
            dff1=dff.groupby(['Month','Platform','mth',d]).agg({kpi:'sum'}).reset_index().sort_values(by='mth')
            fig.add_trace(go.Bar(x=dff1[dff1[d] == 'Yes']['Month'],y=dff1[dff1[d] == 'Yes'][kpi]
                                     ,name=d))

            xaxisTitle='Month'
            titletext=f'Monthly Total {kpi_value}({promo_value})'
        
            kff2 = dff.groupby(['Month']).agg({'Holiday B':'mean'}).reset_index()
            kff3 = dff[dff['Secondary Deli']=='Yes']
            kff4 = dff[dff['Secondary Bakery']=='Yes']
            kff5 = dff[dff['Secondary Center store']=='Yes']
            kff3 = kff3.groupby(['Month']).agg({kpi:'sum'})
            kff4 = kff4.groupby(['Month']).agg({kpi:'sum'})
            kff5 = kff5.groupby(['Month']).agg({kpi:'sum'})
            ymin = min([kff3.min().sum(), kff4.min().sum(), kff5.min().sum()])
            ymax = max([kff3.max().sum(), kff4.max().sum(), kff5.max().sum()])
            
            for i in kff2.index:
                if kff2['Holiday B'][i] > 0:
                    if year_value == 2020:
                        fig.add_shape(type="line",
                                 x0 = kff2['Month'][i],
                                 x1 = kff2['Month'][i],
                                 y0 = ymin,
                                 y1 = ymax*1.1,
                                 line = dict(color="orange",width=140),
                                 opacity=kff2['Holiday B'][i],
                                 layer="below")
                    
                    else:
                        fig.add_shape(type="line",
                                 x0 = kff2['Month'][i],
                                 x1 = kff2['Month'][i],
                                 y0 = ymin,
                                 y1 = ymax*1.1,
                                 line = dict(color="orange",width=80),
                                 opacity=kff2['Holiday B'][i],
                                 layer="below")
                        
    fig.update_layout(
                  yaxis=dict(title=f'<b>{kpi}</b>'),height=500,
    xaxis=dict(title=f'<b>{xaxisTitle}</b>',showgrid=True,showticklabels = True),   

    title={'text':f'<b>{titletext}</b>','xanchor':'center','y':0.85,'x':0.45}
    )
    value_used=0
    for d in displays:
        dfx=df2.groupby(['Year','Platform',d]).agg({kpi:'sum'}).reset_index()
        dfx=dfx[dfx[d] != 'No']    
        fig1.add_trace(go.Bar(x=dfx['Year'],y=dfx[kpi],name=d))
        
    fig1.add_shape(type="rect",
    xref="paper", yref="paper",
    x0=0.70, y0=0,opacity=0.5,
    x1=0.975, y1=1,
    line=dict(
        color="LightSeaGreen",
        width=0,
    ),fillcolor="PaleTurquoise",)
    
    
  
    fig1.update_layout(title={'text':f'<b>Secondary Display Impact</b>',
                                             'xanchor': 'center', 
                                              'yanchor': 'top',
                                              'y':0.85,
                                              'x':0.35},
                      yaxis=dict(title=f'<b>{kpi}</b>'),
                      xaxis=dict(title=f'<b>Year</b>',showgrid=True,showticklabels = True),
                      annotations=[dict(text=f'<b>Only Till July</b>',xref="paper", yref="paper", x=0.95, y=1, font_size=10, showarrow=False)])
                                   
    df3=dff.groupby(['Geography Summarized']).agg({kpi:'sum'}).sort_values(by=kpi,ascending=False)
    df4=df3.head(10)
    df5=df3.reset_index().iloc[11:,]
    df5['Geography Summarized']=['**Others' for i in range(0,len(df5))]
    df6=df5.groupby(['Geography Summarized']).agg({kpi:'sum'}).sort_values(by=kpi,ascending=False)
    df7=df4.append(df6).reset_index()
    df4=df3.reset_index()
    labels = df4['Geography Summarized']
    values = df4[kpi]


    fig2.add_trace(go.Pie(labels=labels, values=values,insidetextorientation='radial', textinfo='label+percent'))
    fig2.update_traces(textposition='inside')
    fig2.update_layout(
#                        uniformtext_minsize=16, 
#                        uniformtext_mode='hide',
                        showlegend=False,
                       title={'text': f'<b>Total {kpi} Split By Retailers</b>',
                               'xanchor': 'center', 'yanchor': 'top','y':0.85,'x':0.5},
                       titlefont=dict(size=14),paper_bgcolor='white',
                      annotations=[dict(text=f'Year:<b>{year_value}</b>', x=0.0, y=1, font_size=10, showarrow=False),
                 dict(text=f'Platform:<b>{platform_value}</b>', x=0.0, y=0.95, font_size=10, showarrow=False)])
    
    return fig,fig1,fig2


if __name__ == "__main__":
    app.run_server(debug=False, port = 1100)

127.0.0.1 - - [17/Nov/2020 20:50:17] "GET /_shutdown_08a0755f-a3e3-4236-881d-8d5b680efa95 HTTP/1.1" 200 -
 * Running on http://127.0.0.1:1100/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2020 20:50:17] "GET /_alive_08a0755f-a3e3-4236-881d-8d5b680efa95 HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:1100/


127.0.0.1 - - [17/Nov/2020 20:50:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:19] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:19] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:19] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:20] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:50:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 21:05:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 21:05:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 21:05:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 21:05:38] "POST 

# Dash Screen Recording Video (please watch video in full screen view for best experience)

In [11]:

# from IPython.core.display import display, HTML
# display(HTML("<style>div.output_scroll { width: 70em; }</style>"))
Video("Dash.mp4",embed=True)

127.0.0.1 - - [17/Nov/2020 20:40:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:40:33] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:40:34] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:40:34] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [17/Nov/2020 20:40:34] "POST /_dash-update-component HTTP/1.1" 200 -
